# Trim LabelStudio annotations

> "to match trimmed audio"

- branch: master
- hidden: true
- comments: false
- categories: [labelstudio, trim, csv]

In [2]:
def slurpfile(filename) -> str:
    with open(filename) as inf:
        return inf.read().strip()

In [3]:
output_dir = "/tmp/textgrid_cut"
host = "http://130.237.3.107:8080/api/"
api_token: str = slurpfile("label_studio_mine")

In [4]:
from pathlib import Path

output_path = Path(output_dir)

In [5]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [6]:
def get_task(task_id):
    ep = f"{host}tasks/{task_id}"
    req = requests.get(ep, headers=headers)
    if req.status_code != 200:
        return {}
    data = json.loads(req.text)
    return data

In [7]:
def get_annotation(annot_it):
    ep = f"{host}annotations/{annot_it}"
    req = requests.get(ep, headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [8]:
data = get_annotation(264)

In [9]:
data

{'id': 264,
 'result': [{'original_length': 341.376,
   'value': {'start': 16.924884235915957,
    'end': 18.024884235915955,
    'channel': 0,
    'text': ['What looks strange?', '/wˌʌts lˈʊks tɹˈeɪndʒ?/']},
   'id': 'KmYVSVjAS5',
   'from_name': 'transcription',
   'to_name': 'audio',
   'type': 'textarea',
   'origin': 'manual'},
  {'original_length': 341.376,
   'value': {'start': 16.924884235915957,
    'end': 18.024884235915955,
    'channel': 0,
    'labels': ['Speech']},
   'id': 'KmYVSVjAS5',
   'from_name': 'labels',
   'to_name': 'audio',
   'type': 'labels',
   'origin': 'manual'},
  {'original_length': 341.376,
   'value': {'start': 19.19713218349019,
    'end': 19.69713218349019,
    'channel': 0,
    'text': ['Fingers.', '/fˈɪŋɡɚz./']},
   'id': 'roPZSFDWDP',
   'from_name': 'transcription',
   'to_name': 'audio',
   'type': 'textarea',
   'origin': 'manual'},
  {'original_length': 341.376,
   'value': {'start': 19.19713218349019,
    'end': 19.69713218349019,
    'chann

In [13]:
get_task(77)

{'id': 77,
 'predictions': [],
 'annotations': [{'id': 153,
   'result': [{'original_length': 341.376,
     'value': {'start': 16.99,
      'end': 17.99,
      'channel': 0,
      'text': ['What looks strange?']},
     'id': 'nJ67iJt0sN',
     'from_name': 'transcription',
     'to_name': 'audio',
     'type': 'textarea',
     'origin': 'manual'},
    {'original_length': 341.376,
     'value': {'start': 19.211,
      'end': 19.632,
      'channel': 0,
      'text': ['Fingers.']},
     'id': 'EwHlDkpXnR',
     'from_name': 'transcription',
     'to_name': 'audio',
     'type': 'textarea',
     'origin': 'manual'},
    {'original_length': 341.376,
     'value': {'start': 25.577,
      'end': 26.798,
      'channel': 0,
      'text': ["But they're all stuck though."]},
     'id': 'PkqrIhzidx',
     'from_name': 'transcription',
     'to_name': 'audio',
     'type': 'textarea',
     'origin': 'manual'},
    {'original_length': 341.376,
     'value': {'start': 27.038,
      'end': 28.259,
 

In [10]:
def combine_labels(data):
    combined = {}

    if "result" in data:
        for res in data["result"]:
            if not res["id"] in combined:
                combined[res["id"]] = res
            else:
                if "text" in res["value"]:
                    combined[res["id"]]["value"]["text"] = res["value"]["text"]
                elif "labels" in res["value"]:
                    combined[res["id"]]["value"]["labels"] = res["value"]["labels"]
    return combined

In [11]:
combined = combine_labels(data)

In [12]:
combined

{'KmYVSVjAS5': {'original_length': 341.376,
  'value': {'start': 16.924884235915957,
   'end': 18.024884235915955,
   'channel': 0,
   'text': ['What looks strange?', '/wˌʌts lˈʊks tɹˈeɪndʒ?/'],
   'labels': ['Speech']},
  'id': 'KmYVSVjAS5',
  'from_name': 'transcription',
  'to_name': 'audio',
  'type': 'textarea',
  'origin': 'manual'},
 'roPZSFDWDP': {'original_length': 341.376,
  'value': {'start': 19.19713218349019,
   'end': 19.69713218349019,
   'channel': 0,
   'text': ['Fingers.', '/fˈɪŋɡɚz./'],
   'labels': ['Speech']},
  'id': 'roPZSFDWDP',
  'from_name': 'transcription',
  'to_name': 'audio',
  'type': 'textarea',
  'origin': 'manual'},
 'TmbbCc257S': {'original_length': 341.376,
  'value': {'start': 25.524884235915955,
   'end': 28.43302462910922,
   'channel': 0,
   'text': ["But they're all stuck though. I mean the things are there.",
    '/bˈʌ ðɛɹ ˈɔːl stˈʌk ðˌoʊ. aɪ mˈiːn ðə θˈɪŋz əɹ ðˈɛɹ./'],
   'labels': ['Speech']},
  'id': 'TmbbCc257S',
  'from_name': 'transcripti

In [58]:
def adjust_times_write_tsv(data):
    task = data["task"]
    task_data = get_task(task)
    if "data" in task_data and "audio" in task_data["data"]:
        orig_file = task_data["data"]["audio"]
        parts = orig_file.split("/")
        orig_file = parts[-1]
    if orig_file:
        out_part = orig_file.replace(".wav", ".tsv")
        orig_file = out_part.replace("_main", "").replace("_inter", "")
    else:
        return []
    out_file = output_path / out_part

    results = []

    combined = combine_labels(data)

    for item in combined:
        val = combined[item]["value"]
        if not "labels" in val:
            continue
        if not "Speech" in val["labels"]:
            continue
        text = val["text"]
        if len(text) > 1:
            for t in text:
                if not (t.startswith("/") and t.endswith("/")):
                    text = t
        else:
            text = text[0]

    with open(out_file, "w") as outf:
        for res in results:
            outf.write("\t".join([str(x) for x in list(res)]) + "\n")
        

In [59]:
for transcription in IDS:
    data = get_annotation(transcription)
    if not "task" in data:
        print("Error with task", transcription)
        continue
    else:
        adjust_times_write_tsv(data)

In [2]:
import yaml

test = {
    "thing": 1,
    "foo": ["a", "b"]
}
print(yaml.dump(test))

foo:
- a
- b
thing: 1

